#### ETL - Data Transformation

@by Sandra Nachforg

#### 1. Import libraries

In [116]:
import tweepy as tw
import re
import pandas as pd
import preprocessor as p
from preprocessor.api import clean, tokenize, parse
import re, string, unicodedata
import nltk
import datetime as dt
import emoji
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('vader_lexicon')
nltk.download('stopwords')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words("english")
from textblob import TextBlob

import requests 
import pandas as pd
import json
from pandas.io.json import json_normalize

[nltk_data] Downloading package wordnet to /Users/sandra/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/sandra/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sandra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


####  We are interested in the sentiment of the following companies :

a) Netflix (hashtag: netflix) \
b) Coca Cola (hashtag: CocaCola) \
c) Tesla (hashtag: Tesla)\
d) Nike (hashtag: Nike) \
e) Apple (hashtag: Apple) 

#### 2. Get data from Alpha Vantage (API that has company information)


In [27]:
key = ""
URL= "https://www.alphavantage.co/query?function=OVERVIEW&symbol=AAPL&apikey=key"

In [30]:
def get_company_data(company,key):
    
    key = key
    URL= "https://www.alphavantage.co/query?function=OVERVIEW"
    r= requests.get(url= URL)
    data = r.json()
    df = json_normalize(data)   #transforms the data into a dataframe
    df  = df[["Name", "Sector", "Industry"]]   #returns only select columns
    
    return df

#### 3. Authorize Account for Twitter

In [117]:
consumer_key= ''
consumer_secret= ''
access_token= ""
access_token_secret= ''
callback_url = 'oob'

auth = tw.OAuthHandler(consumer_key, consumer_secret, callback_url)
redirect_url= auth.get_authorization_url()

print (redirect_url)

auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

user_pin_input = input ("What's the pin value? ")
api = tw.API (auth)

https://api.twitter.com/oauth/authorize?oauth_token=gOK_sQAAAAABNQviAAABeP8xQbM
What's the pin value? 3399798


#### 4. Get function to get data from Twitter API 

In [118]:
#this function takes 3 parameters and helps us extract the data from twitter
# The "hashtag" is the company name we are analyzing
# The "company_id" is the id we assigned to that particular company in the company dimensions
# The "document_id" is the id we assigned to source (Twitter  - 1, Forbes - 2 etc.) 

def get_tweets(hashtag, company_id, document_id):
    
    number_of_tweets = 100
    tweets = []
    likes= []
    time= []
    user = []
    author = []
    retweet=[]
    
    # q = pass the hashtags
    
    for i in tw.Cursor(api.search, q= hashtag, tweet_mode= 'extended', lang= "en").items(number_of_tweets):
        tweets.append(i.full_text)
        likes.append(i.favorite_count)
        time.append(i.created_at)
        retweet.append(i.retweet_count) 
      
        
    df= pd.DataFrame({'tweets': tweets, 'likes': likes, 'timestamp': time, 'retweet': retweet})
    
    df["document_id"] = document_id
    df["company_id"] = company_id
    
    df= df[["company_id", "timestamp", "document_id", "tweets", "retweet", "likes"]]
    
    df.rename(columns = {"timestamp": "time_id", "tweets": "original_description"}, inplace= True)

    return df

#### 5. Create function to get data from News API - can also specify the publisher

In [124]:
"""
@author: Mohamed Abouregila
get_headlines()
 this function takes company, fromDate, and to date in form of  "yyyy-mm-dd", and returns a dataframe of all 
 headlines posted about this company in the specified period

"""

'''Modified: add additional argument: company_id to differentiate from each other - Sandra Nachforg'''

def get_headlines(company_id, company,fromDate,toDate,key):
    
    url= "http://newsapi.org/v2/everything?"
    parameters = {
            "qInTitle": company,
            "language":"en",
            "from":fromDate,
            "to":toDate,
            "apiKey":key,
    }
    
    response = requests.get(url,params=parameters)

    df = response.json()
    df = pd.DataFrame.from_dict(df)
    df = pd.concat([df.drop(['articles'], axis=1), df['articles'].apply(pd.Series)], axis=1)
    df [["Drop","source"]] = df.source.apply(pd.Series)
    df ["company_id"]  = company_id
    df ["document_id"] = 2
    df ["retweet_count"] = ""
    df ["like_count"] = ""
    
    df = df [["company_id", "publishedAt", "document_id", "title", "retweet_count", "like_count"]]
    
    
    df ["publishedAt"] = pd.to_datetime(df_news.publishedAt).dt.tz_localize(None)
    
    df.rename(columns = {"publishedAt": "time_id", "title": "original_description"}, inplace = True)
    
    return df


#### 6. Get tweets for the companies- Apply the "get_tweets()"-  function

This function takes three arguments: hashtag (e.g. netflix), company_id (based on copmany dimension), document_id (based on document dimension)

In [120]:
netflix_twitter = get_tweets("netflix", 1,1)    #netflix, 1 - company_id, 1- document_id 
tesla_twitter = get_tweets("tsla", 2, 1) 
cocacola_twitter = get_tweets("cocacola", 3,1)
starbucks_twitter = get_tweets("starbucks", 4, 1 )
nike_twitter = get_tweets("nike", 5, 1)

#### 7. Get headlines for the companies - Apply the get_headlines () - function

In [125]:
netflix_news = get_headlines (1, "netflix",'2021-04-21','2021-04-22', "" )
tesla_news = get_headlines (2, "tesla", '2021-04-21','2021-04-22', "")
#cocacola_news = get_headlines (3, "cocacola", '2021-04-21','2021-04-22', "") # didn't have source_ 
starbucks_news = get_headlines (4, "starbucks", '2021-04-21','2021-04-22', "")
nike_news = get_headlines (5, "nike", '2021-04-21','2021-04-22', "") 

#### 8. Concat all dataframes (stack on top of each other)

In [126]:
fact_table = pd.concat([netflix_twitter,tesla_twitter, cocacola_twitter, starbucks_twitter, nike_twitter,
                        netflix_news, tesla_news, starbucks_news, nike_news], axis=0)   #stacks each dataframe on top of each other

fact_table.to_csv("fact_table.csv")  #saves fact_table --> in case we want to save a copy to our hard drive

In [131]:
fact_table.head()

,company_id,time_id,document_id,original_description,retweet,likes,retweet_count,like_count
0,1,2021-04-23 14:45:32,1,"Bro, why do we even have a huddle. This shit c...",0.0,0.0,NaN,NaN
1,1,2021-04-23 14:45:32,1,RT @thecartooncrave: Netflix will reportedly s...,15.0,0.0,NaN,NaN
2,1,2021-04-23 14:45:31,1,RT @Rocioceja_: The way y’all canceled so many...,483.0,0.0,NaN,NaN
3,1,2021-04-23 14:45:31,1,@Smileyimxx คูมเจฟท้าเรามาแหละ 3&gt;\n\n𓂋 Netf...,0.0,0.0,NaN,NaN
4,1,2021-04-23 14:45:31,1,"RT @bonobochick: And when I say I have a list,...",1.0,0.0,NaN,NaN


#### 9. Create function to clean original_descriptions


In [132]:
#We need to do this to do sentiment analysis on the headlines/tweets later

def cleanup_description(s):
    text = s.lower().split()       # makes all characters lower case
    text = list(filter(lambda x: "http" not in x, text))        # removes http
    text = list(filter(lambda x: not x.startswith("@"), text))   #removes the @
    text = list(map(lambda x: emoji.demojize(x, delimiters=("", ",")).replace("_", " "), text))  # replaces emojis with word
    text = list(map(lambda x: re.sub("[^a-zA-Z ]+", "", x), text)) 
    #only keeps letter from a-z and A-Z
    #text = [word for word in text if not word in stopwords.words()]     #removes stop words
    lemmatizer = WordNetLemmatizer()
    text = list(map(lambda x: lemmatizer.lemmatize(x), text))     #lemmatizes the text
    
    return " ".join(text)

#### 10. Create a function that calculates the sentiment score for each clean_description

In [133]:
def get_sentiment_score(df):
    
    df["cleaned_description"] =  df["original_description"].apply(lambda x: cleanup_description(x))
    df[["polarity", "subjectivity"]] = df["cleaned_description"].apply(lambda x: pd.Series(TextBlob(x).sentiment))

    for index, row in df["cleaned_description"].iteritems():
        score = SentimentIntensityAnalyzer().polarity_scores(row)
    
        neg = score["neg"]
        neu = score["neu"]
        pos = score["pos"]
        comp = score["compound"]
    
        if neg > pos:
            df.loc[index, "sentiment"] = "negative"
        elif pos > neg:
            df.loc[index, "sentiment"] = "positive"
        else:
            df.loc[index, "sentiment"] = "neutral"
    
    df.rename(columns = {"timestamp": "time_id", "tweets": "original_description", "cleaned_tweets":"cleaned_description", 
                        "retweet": "retweet_count", "likes": "like_count"}, inplace = True)
    
    df = df[["company_id", "time_id", "document_id", "original_description","cleaned_description", "retweet_count", 
            "like_count", "polarity", "subjectivity", "sentiment"]]
    
    
            
    return df

#### 11. After we have our finished dataframe, we can apply  the "get_sentiment_score"- function

In [134]:
get_sentiment_score(fact_table)

,company_id,time_id,document_id,original_description,cleaned_description,retweet_count,retweet_count,like_count,like_count,polarity,subjectivity,sentiment
0,1,2021-04-23 14:45:32,1,"Bro, why do we even have a huddle. This shit c...",bro why do we even have a huddle this shit cou...,0.0,NaN,0.0,NaN,-0.187879,0.751515,neutral
1,1,2021-04-23 14:45:32,1,RT @thecartooncrave: Netflix will reportedly s...,rt netflix will reportedly spend billion on c...,15.0,NaN,0.0,NaN,0.000000,0.000000,positive
2,1,2021-04-23 14:45:31,1,RT @Rocioceja_: The way y’all canceled so many...,rt the way yall canceled so many good as netfl...,483.0,NaN,0.0,NaN,0.300000,0.525000,negative
3,1,2021-04-23 14:45:31,1,@Smileyimxx คูมเจฟท้าเรามาแหละ 3&gt;\n\n𓂋 Netf...,gt netflix or dinner dinner mouse trap kiss...,0.0,NaN,0.0,NaN,0.000000,0.000000,neutral
4,1,2021-04-23 14:45:31,1,"RT @bonobochick: And when I say I have a list,...",rt and when i say i have a list netflix show t...,1.0,NaN,0.0,NaN,0.000000,0.000000,positive
...,...,...,...,...,...,...,...,...,...,...,...,...
15,5,2021-04-22 23:55:45,2,"atmos' Nike LeBron 18 Low ""Sakura"" Nods to Jap...",atmos nike lebron low sakura nod to japan che...,NaN,,NaN,,0.000000,0.300000,negative
16,5,2021-04-22 23:56:03,2,Erik Arteaga Announces Vyzer LA Sponsorship Sk...,erik arteaga announces vyzer la sponsorship sk...,NaN,,NaN,,0.000000,0.000000,neutral
17,5,2021-04-21 11:27:00,2,Take an Official Look At the Nike SB Dunk High...,take an official look at the nike sb dunk high...,NaN,,NaN,,0.160000,0.540000,neutral
18,5,2021-04-21 22:00:00,2,Nike & Urbanstar Team Up With The Silhouettes ...,nike urbanstar team up with the silhouette pr...,NaN,,NaN,,0.000000,0.000000,neutral


#### 12. Lastly - we create time dimension by getting all unique timestamps from fact table and then transform it  
@ by Sandra Nachforg

In [139]:
def get_time_dim(df):
    df_time= df[["time_id"]]
    df_time['Time'] = pd.to_datetime(df['time_id']).dt.time
    df_time['Day']= pd.to_datetime(df['time_id']).dt.day
    df_time['Month']= pd.to_datetime(df['time_id']).dt.month
    df_time['Year']= pd.to_datetime(df['time_id']).dt.year
    
    df_time.drop_duplicates(inplace = True)
    
    return df_time

## timestamp is the primary key here in SQL
## In fact table it is part of the primary key

In [140]:
get_time_dim(fact_table)

<ipython-input-139-a8e35094c473>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_time['Time'] = pd.to_datetime(df['time_id']).dt.time
<ipython-input-139-a8e35094c473>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_time['Day']= pd.to_datetime(df['time_id']).dt.day
<ipython-input-139-a8e35094c473>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

,time_id,Time,Day,Month,Year
0,2021-04-23 14:45:32,14:45:32,23,4,2021
2,2021-04-23 14:45:31,14:45:31,23,4,2021
5,2021-04-23 14:45:30,14:45:30,23,4,2021
7,2021-04-23 14:45:28,14:45:28,23,4,2021
8,2021-04-23 14:45:26,14:45:26,23,4,2021
...,...,...,...,...,...
14,2021-04-22 14:18:31,14:18:31,22,4,2021
15,2021-04-22 23:55:45,23:55:45,22,4,2021
16,2021-04-22 23:56:03,23:56:03,22,4,2021
18,2021-04-21 22:00:00,22:00:00,21,4,2021


In [ ]:
#newspapers= Daily Mail, Extra, Variety, 

In [121]:
!pip install pymysql
import pymysql

  Using cached PyMySQL-1.0.2-py3-none-any.whl (43 kB)


In [122]:
connection = pymysql.connect(host= '127.0.0.1',
                            user = 'root', 
                            password = '',
                            db= '',
                            cursorclass = pymysql.cursors.DictCursor)

print (connection)

In [117]:
from sqlalchemy import create_engine

engine = create_engine('sqlite://', echo=False)

In [118]:
df2 = pd.DataFrame({'name' : ['User 1', 'User 2', 'User 3'], 'age': [23,45,34]})

In [124]:
#df2.to_sql('test', con=engine)

engine.execute("SELECT * FROM test").fetchall()

[(0, 'User 1', 23), (1, 'User 2', 45), (2, 'User 3', 34)]

In [ ]:
sqlQuery = "CREATE TABLE tweets (retweet_count INT, text VARCHAR (1000), author VARCHAR (255), created_at datetime)"

In [ ]:
cursor= connection.cursor()
connection.commit()
sql2 = "show tables;" 
result = cursor.fetchone()

cols = "`,`".join([str(i) for i in df5.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df5.iterrows():
    sqlQuery = "INSERT INTO `tweets` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sqlQuery, tuple(row))

    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()
    
sql = "SELECT * FROM `tweets`"
cursor.execute(sql)

result = cursor.fetchall()
for i in result:
    print(i)